In [3]:
import torch
import tiktoken
enc = tiktoken.get_encoding('gpt2')
enc.n_vocab
enc.encode("hiii there")

[71, 15479, 612]

In [5]:
enc.decode([71, 15479, 612])

'hiii there'

In [18]:
arr = torch.arange(1,11)
blocksize = 8
x = arr[:blocksize]
y = arr[1:blocksize+1]
arr,x, y,arr[blocksize:]

(tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 tensor([1, 2, 3, 4, 5, 6, 7, 8]),
 tensor([2, 3, 4, 5, 6, 7, 8, 9]),
 tensor([ 9, 10]))

In [15]:
x,y 

(tensor([1, 2, 3, 4, 5, 6, 7, 8]), tensor([2, 3, 4, 5, 6, 7, 8, 9]))

In [20]:
for t in range(blocksize):
    print(t)

0
1
2
3
4
5
6
7


In [22]:
range(blocksize)

range(0, 8)

The mathematical trick in self-attention

In [33]:
# conisder the following toy example
torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
# x = torch.randn(B,T,C)
x = torch.arange(B*T*C,dtype=torch.float32)
x = x.view(B,T,C)
x.shape
print(x)

tensor([[[ 0.,  1.],
         [ 2.,  3.],
         [ 4.,  5.],
         [ 6.,  7.],
         [ 8.,  9.],
         [10., 11.],
         [12., 13.],
         [14., 15.]],

        [[16., 17.],
         [18., 19.],
         [20., 21.],
         [22., 23.],
         [24., 25.],
         [26., 27.],
         [28., 29.],
         [30., 31.]],

        [[32., 33.],
         [34., 35.],
         [36., 37.],
         [38., 39.],
         [40., 41.],
         [42., 43.],
         [44., 45.],
         [46., 47.]],

        [[48., 49.],
         [50., 51.],
         [52., 53.],
         [54., 55.],
         [56., 57.],
         [58., 59.],
         [60., 61.],
         [62., 63.]]])


In [34]:
# We want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [45]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
wei
xbow2 = wei @ x # (B{pythorch adds this extra batch},T, T) @ (B, T, C)   -> (B,T,C)
                # it multiplies for each batch element T,T @ T,C
torch.allclose(xbow, xbow2)

True

In [47]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked fill(tril)

tensor(0.)

In [11]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [12]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
b = torch.randint(0,10, (3,2)).float()

c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[ 2.,  7.],
        [ 8., 11.],
        [14., 16.]])


In [14]:
a = a/ torch.sum(a, 1, keepdim=True)


In [15]:
a

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])